# Rede Neural 🧠

Import das bibliotecas necessárias para projeto

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

load_data(filepath): Lê um arquivo excel com os dados.

In [ ]:
def load_data(filepath):
    data = pd.read_excel(filepath, engine='openpyxl')

    return data

prepare_data(data): Prepara os dados para o treinamento do modelo.

In [ ]:
def prepare_data(data):
    X = data.drop(["IPCA", "Selic", "Prefixado"], axis=1).values
    y = data[["IPCA", "Selic", "Prefixado"]].values

    X = X.reshape(-1, 1)
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, scaler

create_model(input_shape): Cria o modelo de rede neural.

In [ ]:
def create_model(input_shape):
    model = keras.models.Sequential([keras.layers.Dense(10, activation="relu", input_shape=input_shape),keras.layers.Dense(3)])
    model.compile(loss="mse", optimizer="sgd", metrics=["mae"])

    return model

train_model(model, X_train, y_train, epochs=100, validation_data=None): Treina o modelo com os dados de treinamento, usando a função de perda mse e o otimizador sgd.

In [ ]:
def train_model(model, X_train, y_train, epochs=100, validation_data=None):
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=validation_data)

    return history

predict_allocation(model, scaler, investimento, X_new): Faz a predição de alocação de investimento para um dado investimento e para um conjunto de dados de entrada X_new.

In [ ]:
def predict_allocation(model, scaler, investimento, X_new):
    X_new_scaled = scaler.transform(X_new)
    y_pred = model.predict(X_new_scaled)
    allocation = investimento * y_pred[0] / y_pred[0].sum()

    return allocation

main(): Função principal que chama as outras funções na ordem correta e imprime os resultados da alocação de investimento.

In [ ]:
def main():
    filepath = "/content/drive/MyDrive/Colab Notebooks/Dados/dados_tesouro_analise.xlsx"
    data = load_data(filepath)


    X_train, X_test, y_train, y_test, scaler = prepare_data(data)
    model = create_model(input_shape=X_train.shape[1:])
    history = train_model(model, X_train, y_train, epochs=100, validation_data=(X_test, y_test))


    investment = 10000

    X_new = np.array([[4.5], [2.25], [5.2]])
    allocations = predict_allocation(model, scaler, investment, X_new)


    print("IPCA = R${:,.2f}".format(allocations[0]))
    print("Selic = R${:,.2f}".format(allocations[1]))
    print("Prefixado = R${:,.2f}".format(allocations[2]))


if __name__ == "__main__":
    main()